IMPORT MODULES

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow.keras import models, layers
import dill

Preprocessing

Save the Image size and the batch size

Then save the preprocessed image as a datafile

Call the categories in the files

In [6]:
IMAGE_SIZE=256
BATCH_SIZE=32
CHANNELS=3
EPOCHS=50

In [7]:

dataset = tf.keras.preprocessing.image_dataset_from_directory('potato', 
                                                    shuffle=True, 
                                                    image_size=(IMAGE_SIZE,IMAGE_SIZE), 
                                                    batch_size=BATCH_SIZE)

Found 2152 files belonging to 3 classes.


In [8]:
categories = dataset.class_names
categories

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [9]:
len(dataset)

68

In [10]:
'''for image_batch, label_batch in dataset.take(1):
    for i in range(12):
        ax=plt.subplot(3,4,i+1)
        plt.imshow(image_batch[i].numpy().astype('uint8'))
        plt.title(categories[label_batch[i]])
        plt.axis('off')'''

"for image_batch, label_batch in dataset.take(1):\n    for i in range(12):\n        ax=plt.subplot(3,4,i+1)\n        plt.imshow(image_batch[i].numpy().astype('uint8'))\n        plt.title(categories[label_batch[i]])\n        plt.axis('off')"

Split data into train, validation and test of 80, 10, 10 respectively

In [11]:
train_size = 0.8
len(dataset)*train_size

54.400000000000006

In [12]:
trained = dataset.take(54)
len(trained)

54

In [13]:
tested = dataset.skip(54)
len(tested)

14

In [14]:
val_size = 0.1
len(dataset)*val_size

6.800000000000001

In [15]:
validated = tested.take(6)
len(validated)

6

In [16]:
tested = tested.skip(6)
len(tested)

8

Write a function to pipeline our train, test, validate, split command

In [17]:
def get_dataset_split(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000 ):
    ds_size = len(ds)

    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    train_size =int(ds_size*train_split)
    val_size = int(ds_size*val_split)
    test_size = int(ds_size*test_split)

    trained = ds.take(train_size)
    validated = ds.skip(train_size).take(val_size)
    test = ds.skip(train_size).skip(val_size)

    return trained, validated, tested 

In [18]:
trained, validated, tested = get_dataset_split(dataset)

In [19]:
len(trained), len(validated), len(tested)

(54, 6, 8)

Cache our datasets to increase efficiency

In [20]:
trained = trained.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
validated = validated.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
tested = tested.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

Setting a resizing class to resize all images to 256*256

Scaling the images using Keras to a number between 0 and 1

In [21]:
resize_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),  
  layers.experimental.preprocessing.Rescaling(1.0/255)  
])

Data Augmentation

In [22]:
augment = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    layers.experimental.preprocessing.RandomRotation(0.2),
])

CNN

Call our model from layers: Resize_Rescale, Augment, Filter, 

In [23]:
input_shape = (BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,CHANNELS)
n_classes=3

model = models.Sequential([
    resize_rescale,
    augment,
    layers.Conv2D(32, (3,3), activation='relu', input_shape = (IMAGE_SIZE,IMAGE_SIZE)),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])
model.build(input_shape=input_shape)

In [24]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 256, 256, 3)         0         
                                                                 
 sequential_1 (Sequential)   (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                  

Compile the model and optimize it

In [25]:
model.compile(
             optimizer='adam',
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
             metrics=['accuracy']
             )

Call fit trained data for the 50 epochs and check for accuracy and loss

In [26]:
history = model.fit(
    trained,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=validated
)

history

Epoch 1/50
54/54 [==============================] - 103s 2s/step - loss: 0.9380 - accuracy: 0.4606 - val_loss: 0.9090 - val_accuracy: 0.5104
Epoch 2/50
54/54 [==============================] - 83s 2s/step - loss: 0.8688 - accuracy: 0.5278 - val_loss: 0.8252 - val_accuracy: 0.5156
Epoch 3/50
54/54 [==============================] - 85s 2s/step - loss: 0.6625 - accuracy: 0.6892 - val_loss: 0.6830 - val_accuracy: 0.6719
Epoch 4/50
54/54 [==============================] - 84s 2s/step - loss: 0.4603 - accuracy: 0.7980 - val_loss: 0.4261 - val_accuracy: 0.8281
Epoch 5/50
54/54 [==============================] - 85s 2s/step - loss: 0.3349 - accuracy: 0.8547 - val_loss: 0.4075 - val_accuracy: 0.8594
Epoch 6/50
54/54 [==============================] - 83s 2s/step - loss: 0.2826 - accuracy: 0.8935 - val_loss: 0.3616 - val_accuracy: 0.8229
Epoch 7/50
54/54 [==============================] - 84s 2s/step - loss: 0.2846 - accuracy: 0.8872 - val_loss: 0.2981 - val_accuracy: 0.8750
Epoch 8/50
54/54 [=

In [2]:
dill.dump_session

NameError: name 'dill' is not defined

In [1]:
dill.load_session

NameError: name 'dill' is not defined

In [27]:
scores = model.evaluate(tested)
scores

8/8 [==============================] - 5s 267ms/step - loss: 0.0124 - accuracy: 0.9957


[0.012427677400410175, 0.9956896305084229]

In [28]:
history

In [29]:
history.params

{'verbose': 1, 'epochs': 50, 'steps': 54}

In [30]:
history.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [31]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

View the Loss and Accuracy on Matplotlib

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(range(EPOCHS), acc, label='Training  Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation  Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

Text(0.5, 1.0, 'Training and Validation Accuracy')

Check the Predictions

In [ ]:
for images_batch, labels_batch in tested.take(1):
    #plt.imshow(images_batch[0].numpy().astype('uint8'))
    first_img = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()
    print('first predicted image')
    
    plt.imshow(first_img)
    print('actual label:', categories[first_label])
    
    batch_prediction = model.predict (images_batch)
    print('predicted label:', categories[np.argmax(batch_prediction[0])])

In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = categories[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class

In [ ]:
plt.figure(figsize=(15,155))
for images, labels in tested.take(1):
    for i in range(9):
        ax = plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype('uint8'))

        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = categories[labels[i]]

        plt.title(f'Actual:{actual_class}, \n Predicted :{predicted_class}')

        plt.axis('off')